# Partie 1

In [314]:
import pandas as pd
import re

def extract_data(input_path, pattern):
    with open(input_path, 'r') as file:
        content = file.read()
    matches = re.findall(pattern, content, re.MULTILINE)
    data = []
    for match in matches:
        numbers = match[1].replace('\n', ' ').split()
        numbers_float = [float(number) for number in numbers]
        data.append(numbers_float)
    return pd.DataFrame(data)

rocks_path = 'data/sonar.rocks'
mines_path = 'data/sonar.mines'


train_pattern = r'\*(CR|CM)\d+:\n\{([\d\s\.\n]+)\}'
test_pattern = r'^(?!\*)(CR|CM)\d+:\n\{([\d\s\.\n]+)\}'


rocks_train_df = extract_data(rocks_path, train_pattern)
mines_train_df = extract_data(mines_path, train_pattern)
rocks_train_df['Label'] = 'R'  
mines_train_df['Label'] = 'M' 
train_df = pd.concat([rocks_train_df, mines_train_df], ignore_index=True).sample(frac=1).reset_index(drop=True)

rocks_test_df = extract_data(rocks_path, test_pattern)
mines_test_df = extract_data(mines_path, test_pattern)
rocks_test_df['Label'] = 'R'  
mines_test_df['Label'] = 'M' 
test_df = pd.concat([rocks_test_df, mines_test_df], ignore_index=True).sample(frac=1).reset_index(drop=True)

## Algorithme d'entraînement

In [315]:
import numpy as np

def minimerror_perceptron(training_features, training_labels, epochs, learning_rate, T_initial, T_decrease_factor):
    
    weights = np.zeros(training_features.shape[1] + 1)
    T = T_initial
    
    for _ in range(epochs):
        
        update = np.zeros(training_features.shape[1] + 1)
        
        for inputs, label in zip(training_features, training_labels):
            
            activation = np.dot(inputs, weights[1:]) + weights[0]
            predicted_label = np.tanh(activation / T)
            
            # Mise à jour de la règle Minimerror, en utilisant la tangente hyperbolique pour la "température"
            error = label - predicted_label
            update[1:] += learning_rate * error * inputs
            update[0] += learning_rate * error
        
        weights += update
        
        # Diminution de la "température" après chaque époque pour affiner l'apprentissage
        T *= T_decrease_factor
    
    return weights

train_df['Label'] = train_df['Label'].apply(lambda x: 1 if x == 'M' else -1)
test_df['Label'] = test_df['Label'].apply(lambda x: 1 if x == 'M' else -1)


train_features = train_df.iloc[:, :-1].values
train_labels = train_df.iloc[:, -1].values
weights = minimerror_perceptron(train_features, train_labels, 600, 0.001, T_initial=80, T_decrease_factor=0.99)
print (weights)


[-0.7258497   0.21159445  0.38491744  0.32606422  0.53436995  0.56897422
  0.21526413  0.05270522 -0.25925507  0.44453582  0.5752605   0.82109167
  0.86641213  0.82248482  0.28308209 -0.28716421 -0.91566584 -0.54622001
 -0.21480961  0.23341056  0.20456587  0.3123386   0.38649658  0.09747376
  0.19790741 -0.14322522 -0.36219101 -0.1941907   0.04100954  0.25692631
  0.18740009 -0.76457626  0.05496442 -0.05310656 -0.41353865 -0.67201051
 -0.86356772 -0.62081879  0.27156894  0.21043719 -0.51119612  0.111341
  0.49329699  0.55662578  0.89377351  1.40883839  1.18875956  0.84899708
  0.75775999  0.56093144  0.04599946  0.09881678  0.09159663  0.02209409
  0.05353741 -0.00682674  0.00601123 -0.0147758   0.04354615  0.03551326
  0.0123419 ]


Ea et Eg

In [316]:
def predict_perceptron(inputs, weights):
    activation = np.dot(inputs, weights[1:]) + weights[0]
    return 1 if activation >= 0 else -1


def calculate_accuracy(features, labels, weights):
    predictions = [predict_perceptron(x, weights) for x in features]
    correct_predictions = sum(pred == label for pred, label in zip(predictions, labels))
    accuracy = correct_predictions / len(labels)
    return accuracy


test_features = test_df.iloc[:, :-1].values
test_labels = test_df.iloc[:, -1].values


training_accuracy = calculate_accuracy(train_features, train_labels, weights)
testing_accuracy = calculate_accuracy(test_features, test_labels, weights)


Ea = 1 - training_accuracy
Eg = 1 - testing_accuracy

print(f"Erreur d'apprentissage (Ea) : {Ea}")
print(f"Erreur de généralisation (Eg) : {Eg}")


Erreur d'apprentissage (Ea) : 0.1923076923076923
Erreur de généralisation (Eg) : 0.14423076923076927


2 attempt

In [317]:
import numpy as np

def minimerror_perceptron2(training_features, training_labels, epochs, learning_rate, T_initial, T_decrease_factor):
    # Initialisation des poids avec une petite valeur non nulle pour éviter la division par zéro.
    weights = np.random.uniform(low=-0.5, high=0.5, size=training_features.shape[1] + 1)
    T = T_initial  # Température initiale

    for epoch in range(epochs):
        for inputs, label in zip(training_features, training_labels):
            # Calcul de la stabilité pour l'exemple actuel
            gamma = label * (np.dot(inputs, weights[1:]) + weights[0])
            
            # Calcul de la contribution à la fonction de coût pour cet exemple
            V = 0.5 * (1 - np.tanh(gamma / (2 * T)))
            
            # Calcul de la dérivée de V par rapport à gamma
            dV_dgamma = -0.5 * (1 - np.tanh(gamma / (2 * T))**2) * (1 / (2 * T))
            
            # Mise à jour des poids - la descente de gradient sur la fonction de coût
            weights[1:] -= learning_rate * dV_dgamma * label * inputs
            weights[0] -= learning_rate * dV_dgamma * label
        
        # Diminution de la température après chaque époque pour affiner l'apprentissage
        T *= T_decrease_factor
    
    return weights

# Paramètres de l'algorithme (à ajuster selon le problème)
epochs = 1000
learning_rate = 0.001
T_initial = 100.0  # Température initiale élevée
T_decrease_factor = 0.99  # Facteur de diminution de la température

weights2 = minimerror_perceptron2(train_features, train_labels, epochs, learning_rate, T_initial, T_decrease_factor)


Ea et Eg 2

In [318]:

training_accuracy2 = calculate_accuracy(train_features, train_labels, weights2)
testing_accuracy2 = calculate_accuracy(test_features, test_labels, weights2)


Ea2 = 1 - training_accuracy2
Eg2 = 1 - testing_accuracy2

print(f"Erreur d'apprentissage (Ea) : {Ea2}")
print(f"Erreur de généralisation (Eg) : {Eg2}")

Erreur d'apprentissage (Ea) : 0.15384615384615385
Erreur de généralisation (Eg) : 0.17307692307692313


3

In [319]:
import numpy as np

def calculate_error(weights, training_features, training_labels):
    errors = 0
    for inputs, label in zip(training_features, training_labels):
        inputs_with_bias = np.insert(inputs, 0, 1)
        prediction = np.sign(np.dot(weights, inputs_with_bias))
        if prediction != label:
            errors += 1
    return errors

weightsT = np.random.uniform(low=0.3, high=0.4, size=train_features.shape[1] + 1)
print(weightsT) 

def minimerror_perceptron(weightsT,training_features, training_labels, epochs, learning_rate, T_initial, T_decrease_factor, patience=200):
    T = T_initial
    weights = weightsT
    previous_errors = float('inf')
    no_improvement_count = 0  # Compter le nombre d'époques sans amélioration

    for epoch in range(epochs):
        for inputs, label in zip(training_features, training_labels):
            inputs_with_bias = np.insert(inputs, 0, 1)
            stability = (np.dot(inputs_with_bias, weights) * label) / np.linalg.norm(weights[1:])
            derivative = -inputs_with_bias * label / (4 * T * np.cosh(stability / (2 * T))**2)
            weights[1:] -= learning_rate * derivative[1:] / np.linalg.norm(weights[1:])
            weights[0] -= learning_rate * derivative[0]

        current_errors = calculate_error(weights, training_features, training_labels)
        
        if current_errors < previous_errors:
            previous_errors = current_errors
            no_improvement_count = 0  # Réinitialiser le compteur car il y a eu amélioration
            weightsT = weights
        else:
            no_improvement_count += 1  # Incrémenter le compteur d'absence d'amélioration

        # Arrêter si aucune amélioration n'a été constatée après un certain nombre d'époques ('patience')
        if no_improvement_count >= patience:
            print(f'Arrêt prématuré après {epoch+1} époques en raison de l\'absence d\'amélioration.')
            break

        # Diminution de la température T pour l'itération suivante
        T *= T_decrease_factor

        # Informations de débogage
        print(f'Epoch {epoch+1}/{epochs}, Erreurs: {current_errors}, Température: {T}')

    return weightsT

# Exemple d'utilisation de la méthode minimerror_perceptron:
weights3 = minimerror_perceptron(weightsT,train_features, train_labels, epochs=10000, learning_rate=0.02, T_initial=80, T_decrease_factor=0.99, patience=500)

training_accuracy3 = calculate_accuracy(train_features, train_labels, weights3)
testing_accuracy3 = calculate_accuracy(test_features, test_labels, weights3)


Ea3 = 1 - training_accuracy3
Eg3 = 1 - testing_accuracy3

print(f"Erreur d'apprentissage (Ea) : {Ea3}")
print(f"Erreur de généralisation (Eg) : {Eg3}")

[0.30083975 0.38771383 0.33076397 0.32051085 0.33517944 0.3637854
 0.33839798 0.33474754 0.33263978 0.36444387 0.31817692 0.30541673
 0.37403221 0.39977461 0.34124352 0.32988372 0.32484278 0.31173699
 0.38935266 0.36340222 0.33344302 0.34910158 0.35973253 0.33834566
 0.37217025 0.35412521 0.30395982 0.39870506 0.39480392 0.38524662
 0.39099991 0.32006311 0.39380766 0.389553   0.38921809 0.33237979
 0.31393129 0.33579676 0.32136142 0.33459248 0.32697495 0.36813902
 0.33051695 0.36957772 0.30510319 0.35775836 0.35903259 0.30469198
 0.37858276 0.34227929 0.33403079 0.38635638 0.36563891 0.35838139
 0.30841203 0.35826889 0.33434125 0.34780601 0.3962127  0.34772427
 0.35982393]
Epoch 1/10000, Erreurs: 55, Température: 79.2
Epoch 2/10000, Erreurs: 55, Température: 78.408
Epoch 3/10000, Erreurs: 55, Température: 77.62392
Epoch 4/10000, Erreurs: 55, Température: 76.84768079999999
Epoch 5/10000, Erreurs: 55, Température: 76.07920399199999
Epoch 6/10000, Erreurs: 55, Température: 75.318411952079

C:\Users\Mustapha\AppData\Local\Temp\ipykernel_2772\715955640.py:25: RuntimeWarning: overflow encountered in scalar power
  derivative = -inputs_with_bias * label / (4 * T * np.cosh(stability / (2 * T))**2)


Epoch 1143/10000, Erreurs: 15, Température: 0.0008205733572556536
Epoch 1144/10000, Erreurs: 15, Température: 0.0008123676236830971
Epoch 1145/10000, Erreurs: 15, Température: 0.0008042439474462661
Epoch 1146/10000, Erreurs: 15, Température: 0.0007962015079718035
Epoch 1147/10000, Erreurs: 15, Température: 0.0007882394928920855
Epoch 1148/10000, Erreurs: 15, Température: 0.0007803570979631646
Epoch 1149/10000, Erreurs: 15, Température: 0.0007725535269835329
Epoch 1150/10000, Erreurs: 15, Température: 0.0007648279917136975
Epoch 1151/10000, Erreurs: 15, Température: 0.0007571797117965606
Epoch 1152/10000, Erreurs: 15, Température: 0.0007496079146785949
Epoch 1153/10000, Erreurs: 15, Température: 0.0007421118355318089
Epoch 1154/10000, Erreurs: 15, Température: 0.0007346907171764908
Epoch 1155/10000, Erreurs: 15, Température: 0.0007273438100047259
Epoch 1156/10000, Erreurs: 15, Température: 0.0007200703719046787
Epoch 1157/10000, Erreurs: 15, Température: 0.0007128696681856319
Epoch 1158

C:\Users\Mustapha\AppData\Local\Temp\ipykernel_2772\715955640.py:25: RuntimeWarning: overflow encountered in cosh
  derivative = -inputs_with_bias * label / (4 * T * np.cosh(stability / (2 * T))**2)


Epoch 1208/10000, Erreurs: 15, Température: 0.00042697756958732337
Epoch 1209/10000, Erreurs: 15, Température: 0.0004227077938914501
Epoch 1210/10000, Erreurs: 15, Température: 0.0004184807159525356
Epoch 1211/10000, Erreurs: 15, Température: 0.0004142959087930102
Epoch 1212/10000, Erreurs: 15, Température: 0.0004101529497050801
Epoch 1213/10000, Erreurs: 15, Température: 0.0004060514202080293
Epoch 1214/10000, Erreurs: 15, Température: 0.000401990906005949
Epoch 1215/10000, Erreurs: 15, Température: 0.0003979709969458895
Epoch 1216/10000, Erreurs: 15, Température: 0.0003939912869764306
Epoch 1217/10000, Erreurs: 15, Température: 0.00039005137410666626
Epoch 1218/10000, Erreurs: 15, Température: 0.0003861508603655996
Epoch 1219/10000, Erreurs: 15, Température: 0.0003822893517619436
Epoch 1220/10000, Erreurs: 15, Température: 0.00037846645824432415
Epoch 1221/10000, Erreurs: 15, Température: 0.0003746817936618809
Epoch 1222/10000, Erreurs: 15, Température: 0.0003709349757252621
Epoch 12